In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
# Install versi yang kompatibel di Colab
%pip install mediapipe==0.10.8 protobuf==3.20.3 numpy==1.26.4

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import cv2
import numpy as np
import pandas as pd
import mediapipe as mp
from tqdm import tqdm

DATASET_PATH = 'D:\Semester 6\PEMROGRAMAN SISTEM CERDAS 2\capstone_mp_model\data'  # Ganti sesuai foldermu
MAX_FRAMES = 30

In [4]:
mp_pose = mp.solutions.pose

def extract_keypoints_from_video(video_path, max_frames=MAX_FRAMES):
    cap = cv2.VideoCapture(video_path)
    pose = mp_pose.Pose()
    sequence = []

    while cap.isOpened() and len(sequence) < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        if results.pose_landmarks:
            keypoints = []
            for lm in results.pose_landmarks.landmark:
                keypoints.extend([lm.x, lm.y])  # Tambah lm.z jika butuh 3D
            sequence.append(keypoints)

    cap.release()
    pose.close()

    if len(sequence) == max_frames:
        return np.array(sequence).flatten()
    return None

In [5]:
all_data = []
all_labels = []

class_names = sorted(os.listdir(DATASET_PATH))

for label in class_names:
    class_path = os.path.join(DATASET_PATH, label)
    if os.path.isdir(class_path):
        for video_file in tqdm(os.listdir(class_path), desc=f"Proses kelas '{label}'"):
            if video_file.endswith(('.mp4', '.avi', '.mov')):
                video_path = os.path.join(class_path, video_file)
                features = extract_keypoints_from_video(video_path)
                if features is not None:
                    all_data.append(features)
                    all_labels.append(label)

# Simpan CSV
df = pd.DataFrame(all_data)
df["label"] = all_labels
df.to_csv("D:\Semester 6\PEMROGRAMAN SISTEM CERDAS 2\capstone_mp_model\Model tflite\pose_dataset.csv", index=False)
print("pose_dataset.csv berhasil disimpan")

Proses kelas 'Wrist Stretch': 100%|██████████| 1/1 [00:01<00:00,  1.44s/it]


pose_dataset.csv berhasil disimpan


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Load CSV
df = pd.read_csv("D:\Semester 6\PEMROGRAMAN SISTEM CERDAS 2\capstone_mp_model\Model tflite\pose_dataset.csv")

X = df.drop("label", axis=1).values
y = df["label"]

# Encode label
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_cat = to_categorical(y_encoded)

# Reshape input
X = X.reshape((X.shape[0], MAX_FRAMES, -1)).astype("float32")

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

# Buat model
model = Sequential()
model.add(LSTM(64, input_shape=(MAX_FRAMES, X.shape[2])))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_cat.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Training
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))


d:\Semester 6\program\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        33,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │           975 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,671 (151.06 KB)

 Trainable params: 38,671 (151.06 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 480ms/step - accuracy: 0.1157 - loss: 2.6839 - val_accuracy: 0.0000e+00 - val_loss: 2.9569
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - accuracy: 0.1157 - loss: 2.5945 - val_accuracy: 0.0000e+00 - val_loss: 3.0599
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.1736 - loss: 2.6513 - val_accuracy: 0.0000e+00 - val_loss: 3.1724
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.0741 - loss: 2.5763 - val_accuracy: 0.0000e+00 - val_loss: 3.3051
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - accuracy: 0.2315 - loss: 2.5046 - val_accuracy: 0.0000e+00 - val_loss: 3.4402
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 0.2685 - loss: 2.4642 - val_accuracy: 0.0000e+00 - val_loss: 3.5475
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.0579 - loss: 2.5410 - val_accuracy: 0.0000e+00 - val_loss: 3.6225
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.2894 - loss: 2.4758 - val_accura

In [7]:
# Simpan model H5
model.save("D:\Semester 6\PEMROGRAMAN SISTEM CERDAS 2\capstone_mp_model\Model tflite\pose_lstm_model.h5")

In [8]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.experimental_enable_resource_variables = True

# Gunakan ops tambahan
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,     # Untuk operasi standar
    tf.lite.OpsSet.SELECT_TF_OPS        # Untuk operasi TensorFlow yang belum didukung penuh TFLite
]

# Jangan turunkan TensorList ops (biarkan dalam bentuk aslinya)
converter._experimental_lower_tensor_list_ops = False

tflite_model = converter.convert()

with open("D:\Semester 6\PEMROGRAMAN SISTEM CERDAS 2\capstone_mp_model\Model tflite\pose_lstm_model.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ Berhasil dikonversi ke TFLite dengan SELECT_TF_OPS.")

INFO:tensorflow:Assets written to: C:\Users\skymo\AppData\Local\Temp\tmpklyfzqr4\assets


INFO:tensorflow:Assets written to: C:\Users\skymo\AppData\Local\Temp\tmpklyfzqr4\assets


Saved artifact at 'C:\Users\skymo\AppData\Local\Temp\tmpklyfzqr4'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 30, 66), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 15), dtype=tf.float32, name=None)
Captures:
  1519495196768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1519495199584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1519495199760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1519486216928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1519486217632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1519486221856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1519485930128: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ Berhasil dikonversi ke TFLite dengan SELECT_TF_OPS.
